# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv('city.csv')
file.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sokol,59.4617,40.1206,25.11,92,91,15.05,RU,1648339856
1,1,Bluff,-46.6000,168.3333,58.62,73,59,2.06,NZ,1648339693
2,2,Tomatlán,19.9333,-105.2500,82.02,33,8,7.05,MX,1648339857
3,3,Esperance,-33.8667,121.9000,66.04,85,100,13.91,AU,1648339815
4,4,Korla,41.7597,86.1469,43.41,53,79,3.06,CN,1648339750


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure the gmaps with the gmap api key
gmaps.configure(api_key=g_key)

In [4]:
#use the lat and lng as locations and the humidity as weight from the file for the locations
#used 06-03-07 as example for heatmap
locations=file[["Lat", "Lng"]]

humidity= file["Humidity"].astype(float)

In [35]:
#create figure for map set zoom as 12,0 to make it show more northern points, and set zoom level to show one map only.
fig = gmaps.figure(center=[12,0], zoom_level=1.95)
#added heatmap layer, and created a weight based on the humidity, and the intensity based on 100, and the point radius to
#give more of a heat map look, vs tiny points
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius =3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
#weather conditions are as follows: max temp 85-75, wind speed less than 10, no clouds
#create a weather condition for the temperature to create a data frame within that request, then call the weather condition
#to the wind condition, and lastly call the wind condition with no clouds to get a list of locations that meet all three of
#the criteria requested
weatherconditions=file.loc[(file['Max Temp']>=75) & (file["Max Temp"]<85)]
windconditions=weatherconditions.loc[(weatherconditions["Wind Speed"]<=10)]
bestweather=windconditions.loc[(windconditions['Cloudiness']==0)]
bestweather.head(20)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
128,128,Arni,12.6667,79.2833,76.53,90,0,5.35,IN,1648339882
146,146,Ukiah,39.1502,-123.2078,75.43,49,0,8.05,US,1648339886
246,246,Jiwani,25.0500,61.7417,77.09,68,0,6.98,PK,1648339915
421,421,Lamar,33.6668,-95.5836,75.11,31,0,5.75,US,1648339958
448,448,Saint-Pierre,-21.3393,55.4781,76.68,72,0,6.91,RE,1648339721
496,496,Qui Nhon,13.7667,109.2333,79.21,80,0,3.56,VN,1648339979
498,498,Sur,22.5667,59.5289,81.90,34,0,6.04,OM,1648339737
532,532,Turbat,26.0023,63.0440,76.57,20,0,2.06,PK,1648339988
565,565,Alamogordo,32.8995,-105.9603,80.22,9,0,5.75,US,1648339997


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
hotel_df=bestweather.drop(columns=['Unnamed: 0']).reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Arni,12.6667,79.2833,76.53,90,0,5.35,IN,1648339882
1,Ukiah,39.1502,-123.2078,75.43,49,0,8.05,US,1648339886
2,Jiwani,25.0500,61.7417,77.09,68,0,6.98,PK,1648339915
3,Lamar,33.6668,-95.5836,75.11,31,0,5.75,US,1648339958
4,Saint-Pierre,-21.3393,55.4781,76.68,72,0,6.91,RE,1648339721
5,Qui Nhon,13.7667,109.2333,79.21,80,0,3.56,VN,1648339979
6,Sur,22.5667,59.5289,81.90,34,0,6.04,OM,1648339737
7,Turbat,26.0023,63.0440,76.57,20,0,2.06,PK,1648339988
8,Alamogordo,32.8995,-105.9603,80.22,9,0,5.75,US,1648339997


In [49]:
#add hotel named column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Arni,12.6667,79.2833,76.53,90,0,5.35,IN,1648339882,
1,Ukiah,39.1502,-123.2078,75.43,49,0,8.05,US,1648339886,
2,Jiwani,25.0500,61.7417,77.09,68,0,6.98,PK,1648339915,
3,Lamar,33.6668,-95.5836,75.11,31,0,5.75,US,1648339958,
4,Saint-Pierre,-21.3393,55.4781,76.68,72,0,6.91,RE,1648339721,
5,Qui Nhon,13.7667,109.2333,79.21,80,0,3.56,VN,1648339979,
6,Sur,22.5667,59.5289,81.90,34,0,6.04,OM,1648339737,
7,Turbat,26.0023,63.0440,76.57,20,0,2.06,PK,1648339988,
8,Alamogordo,32.8995,-105.9603,80.22,9,0,5.75,US,1648339997,


In [50]:
#adding params for the searching of hotels
#ref 06-03-05
params = {
    "radius":5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
    
}


In [56]:
#create a search based on the lat and lng from the dataframe
#import json to use it to print the names in json
import json
#using the lat and lng to identify hotels nearby
for index, row in hotel_df.iterrows():
# getting the lat and long from the df
    lat = row["Lat"]
    lng = row["Lng"]
    #changing location for each iteration while leaving the original params in place
    params['location'] = f"{lat},{lng}"
    #create the base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #makes a request and then gets the url based on the params
    hotel_name = requests.get(base_url, params=params)
    #converst the name to json
    hotel_name=hotel_name.json()
    #prints the json remove comment below to see all of the details for the hotels
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    #adding a try so if there is any missing results it will print out it is missing
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name['results'][0]['name']
    except(KeyError, IndexError):
            print("Missing Field or Missing Result ... Skipping...")

In [54]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Arni,12.6667,79.2833,76.53,90,0,5.35,IN,1648339882,OYO 72131 Nts Residency
1,Ukiah,39.1502,-123.2078,75.43,49,0,8.05,US,1648339886,Hampton Inn Ukiah
2,Jiwani,25.0500,61.7417,77.09,68,0,6.98,PK,1648339915,Hamza hotel jiwani
3,Lamar,33.6668,-95.5836,75.11,31,0,5.75,US,1648339958,Days Inn by Wyndham Paris
4,Saint-Pierre,-21.3393,55.4781,76.68,72,0,6.91,RE,1648339721,Le Battant Des Lames


In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))